In [3]:
import json
from pprint import pprint
import plotly.express as px
import pandas as pd
import chart_studio.plotly as csp
from analytics.prov import ProvenanceAnalyser


In [4]:
import json
analysis_results = json.load(open('provenance_analysis_results.json', 'r', encoding='utf-8'))

# # If the file's content is not already sorted, sort it as follows (the dictionary must be sorted for the visualization to work properly)
# analysis_results = ProvenanceAnalyser.sort_prov_analysis_results(analysis_results)

In [5]:
pprint(analysis_results)


{'': {'https://api.crossref.org/': {'omid_only': 356955, 'other_pids': 324066},
      'https://api.crossref.org/snapshots/monthly/2023/09/all.json.tar.gz': {'omid_only': 71227,
                                                                             'other_pids': 149362},
      'https://api.datacite.org/': {'omid_only': 9, 'other_pids': 162061},
      'https://api.datacite.org/ https://api.crossref.org/': {'omid_only': 0,
                                                              'other_pids': 287},
      'https://doi.org/10.5281/zenodo.7845968': {'omid_only': 0,
                                                 'other_pids': 355075},
      'https://doi.org/10.5281/zenodo.7845968 https://api.crossref.org/': {'omid_only': 0,
                                                                           'other_pids': 16},
      'https://nih.figshare.com/collections/iCite_Database_Snapshots_NIH_Open_Citation_Collection_/4586573/42': {'omid_only': 1,
                                     

In [103]:
# count all the values in the nested dictionaries
total = 0
for k_type, v_source in analysis_results.items():
    for k1, v1 in v_source.items():
        total += sum(v1.values())
print(total)



18131618


In [104]:

# Create a DataFrame to make it easier to work with the data
df = pd.DataFrame(analysis_results).T  # Transpose the DataFrame to have 'br types' as columns

# Add new columns for sum of 'omid_only' and 'other_pids'
df['omid_only_sum'] = df.apply(lambda row: sum(item.get('omid_only', 0) if isinstance(item, dict) else 0 for item in row), axis=1)
df['other_pids_sum'] = df.apply(lambda row: sum(item.get('other_pids', 0) if isinstance(item, dict) else 0 for item in row), axis=1)

# Reset the index to have 'br types' as a regular column
df.reset_index(inplace=True)
df.rename(columns={'index': 'BR Type'}, inplace=True)
df['BR Type'].replace('', 'Unknown', inplace=True)  # Replace empty type string with 'Unknown'

# Create a new column for the legend labels
df['Legend Label'] = df['BR Type'] + ' (' + (df['omid_only_sum'] + df['other_pids_sum']).astype(str) + ')'


# Create the bar chart
fig = px.bar(df, x='BR Type', y=['omid_only_sum', 'other_pids_sum'], color='Legend Label',
             labels={'Number of BRs': ['omid_only_sum', 'other_pids_sum']},
             title='Number of non-mapped BRs per BR Type',
             hover_name='Legend Label',
             # custom_data=df[df.columns[1:13]].fillna('0')
             )

# Customize the layout
fig.update_layout(xaxis_title='BR Type', yaxis_title='Number of BRs')



fig.show()

In [10]:



# Create a DataFrame to make it easier to work with the data
df = pd.DataFrame(analysis_results).T  # Transpose the DataFrame to have 'br types' as columns

# Add new columns for sum of 'omid_only' and 'other_pids'
df['omid_only_sum'] = df.apply(lambda row: sum(item.get('omid_only', 0) if isinstance(item, dict) else 0 for item in row), axis=1)
df['other_pids_sum'] = df.apply(lambda row: sum(item.get('other_pids', 0) if isinstance(item, dict) else 0 for item in row), axis=1)
# Add new column for sum of all values
df['Number of BRs'] = df['omid_only_sum'] + df['other_pids_sum']


# Reset the index to have 'br types' as a regular column
df.reset_index(inplace=True)
df.rename(columns={'index': 'BR Type'}, inplace=True)
df['BR Type'].replace('', 'Unknown', inplace=True)  # Replace empty type string with 'Unknown'

# Create a new column for the legend labels
df['Legend Label'] = df['BR Type'] + ' (' + df['Number of BRs'].astype(str) + ')'


# Create the bar chart
fig = px.bar(df, x='BR Type', y='Number of BRs', text='Number of BRs', color='Legend Label',
             # labels={'Number of BRs': 'Number of BRs', 'omid_only_sum': 'Omid Only', 'With other PIDs': 'other_pids_sum'},
             title='Number of non-mapped BRs per BR Type (with provenance information)',
             hover_name='Legend Label',
             hover_data=['omid_only_sum', 'other_pids_sum'],
             # custom_data=df[df.columns[1:13]].fillna('0')
             )

# Customize the layout
fig.update_layout(xaxis_title='BR Type', yaxis_title='Number of BRs')

# # display hover info for the sum of values in nested dictionaries
# 
# hover_template = '<b>%{hovertext}</b><br><br>' + \
#                  'Number of BRs: %{y}<br><br>' + \
#                  '<b>Provenance Data</b><br>' + \
#                  'Crossref: %{customdata[0]}<br>' + \
#                  'Zenodo: %{customdata[1]}<br>' + \
#                  'Pubmed: %{customdata[2]}<br>' + \
#                  'Zenodo + Crossref: %{customdata[3]}<br>' + \
#                  'Datacite + Crossref: %{customdata[4]}<br>' + \
#                  'Zenodo + Pubmed: %{customdata[5]}<br>' + \
#                  'Pubmed + Crossref: %{customdata[6]}<br>' + \
#                  'Zenodo + Crossref + PubMed: %{customdata[7]}<br>' + \
#                     'Datacite: %{customdata[8]}<br>' + \
#                     'Crossref + Zenodo + Datacite: %{customdata[9]}<br>' + \
#                     'Datacite + Zenodo: %{customdata[10]}<br>' + \
#                     'Zenodo + Datacite + PubMed: %{customdata[11]}<br>'
# 
# fig.update_traces(hovertemplate=hover_template)


fig.show()


In [17]:
import pandas as pd
import plotly.express as px


# Create a DataFrame
df = pd.DataFrame(analysis_results).T
df['omid_only_sum'] = df.apply(lambda row: sum(item.get('omid_only', 0) if isinstance(item, dict) else 0 for item in row), axis=1)
df['other_pids_sum'] = df.apply(lambda row: sum(item.get('other_pids', 0) if isinstance(item, dict) else 0 for item in row), axis=1)
df['Number of BRs'] = df['omid_only_sum'] + df['other_pids_sum']



# Reset the index to have 'br types' as a regular column
df.reset_index(inplace=True)
df.rename(columns={'index': 'BR Type'}, inplace=True)
df['BR Type'].replace('', 'Unknown', inplace=True)  # Replace empty type string with 'Unknown'

# Create a new column for the legend labels
df['Legend Label'] = df['BR Type'] + ' (' + df['Number of BRs'].astype(str) + ')'


# Create the bar chart
fig = px.bar(df, x='BR Type', y='Number of BRs', text='Number of BRs', color='Legend Label',
             # labels={'Number of BRs': 'Number of BRs', 'omid_only_sum': 'Omid Only', 'With other PIDs': 'other_pids_sum'},
             title='Number of non-mapped BRs per BR Type (with provenance information)',
             hover_name='Legend Label',
             hover_data=['omid_only_sum', 'other_pids_sum'],
             # custom_data=df[df.columns[1:13]].fillna('0')
             )

# Customize the layout
fig.update_layout(xaxis_title='BR Type', yaxis_title='Number of BRs')


In [108]:

def get_tot_contribution_by_source(sources_for_type:dict):
    
    res = dict()
    for k, v in sources_for_type.items():
        if ' ' not in k:
            if k in res:
                res[k] += v
            else:
                res[k] = v
        else:
            for single_source in k.split():
                if single_source in res:
                    res[single_source] += v
                else:
                    res[single_source] = v
    return res


In [109]:
ji = sorted_res['journal issue']
ji_by_source = get_tot_contribution_by_source(sorted_res['journal issue'])
pprint(ji)
print('\n')
pprint(ji_by_source)

NameError: name 'sorted_res' is not defined

In [ ]:
import plotly.express as px


def visualize_sources_proportion(sources_for_type:dict, title:str):
    """
    Shows a pie chart of the proportion of contributions from each single source for a given type of bibliographic resource. Each single source's contribution is the results from the sum of all the bibliographic entities that have the source in their provenance data, therefore it is not necessarily a unique contribution, but rather it includes both the number of resources that are provided exclusively by the source and the number of resources that are provided by the source and other sources.
    :param sources_for_type: 
    :param title: 
    :return: 
    """
    data = get_tot_contribution_by_source(sources_for_type)
    labels = list(data.keys())
    values = list(data.values())

    fig = px.pie(values=values, names=labels, title=title)
    fig.show()
    csp.plot(fig, filename=title, auto_open=False, sharing='public')

In [ ]:
visualize_sources_proportion(sorted_res['journal issue'], 'Journal Issue')
visualize_sources_proportion(sorted_res['journal volume'], 'Journal Volume')
visualize_sources_proportion(sorted_res['dataset'], 'Dataset')
visualize_sources_proportion(sorted_res[''], 'Unknown')
visualize_sources_proportion(sorted_res['journal article'], 'Journal Article')

In [ ]:
visualize_sources_proportion(sorted_res['journal'], 'Journal')

In [ ]:
assert False
for k, v in sorted_res.items():
    visualize_sources_proportion(v, k)